In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer

Amazon Sale Report cleaning

In [ ]:
ds = pd.read_csv("/content/Amazon Sale Report.csv")
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   index                   128975 non-null  int64  
 1   Order_ID                128975 non-null  object 
 2   Order_Date              128975 non-null  object 
 3   Order_Status            128975 non-null  object 
 4   Fulfillment_Type        128975 non-null  object 
 5   Sales_Channel           128975 non-null  object 
 6   Shipping_Service_Level  128975 non-null  object 
 7   Product_Style           128975 non-null  object 
 8   Product_SKU             128975 non-null  object 
 9   Product_Category        128975 non-null  object 
 10  Product_Size            128975 non-null  object 
 11  Amazon_Standard_ID      128975 non-null  object 
 12  Courier_Status          122103 non-null  object 
 13  Quantity                128975 non-null  int64  
 14  Currency            

In [ ]:
ds['Shipping_State'] = ds['Shipping_State'].str.upper()

state_mapping = {
    'RJ': 'RAJASTHAN',
    'NL': 'NAGALAND',
    'AR': 'ARUNACHAL PRADESH',
    'PB': 'PUNJAB',
    'ORISSA':'ODISHA',
    'PUNJAB/MOHALI/ZIRAKPUR': 'PUNJAB'
}
ds['Shipping_State'] = ds['Shipping_State'].replace(state_mapping)

states = ds['Shipping_State'].unique()
print("States: ", states)

States:  ['MAHARASHTRA' 'KARNATAKA' 'PUDUCHERRY' 'TAMIL NADU' 'UTTAR PRADESH'
 'CHANDIGARH' 'TELANGANA' 'ANDHRA PRADESH' 'RAJASTHAN' 'DELHI' 'HARYANA'
 'ASSAM' 'JHARKHAND' 'CHHATTISGARH' 'ODISHA' 'KERALA' 'MADHYA PRADESH'
 'WEST BENGAL' 'NAGALAND' 'GUJARAT' 'UTTARAKHAND' 'BIHAR'
 'JAMMU & KASHMIR' 'PUNJAB' 'HIMACHAL PRADESH' 'ARUNACHAL PRADESH'
 'MANIPUR' 'GOA' 'MEGHALAYA' 'TRIPURA' 'LADAKH' 'DADRA AND NAGAR' 'SIKKIM'
 nan 'ANDAMAN & NICOBAR ' 'RAJSHTHAN' 'MIZORAM' 'NEW DELHI' 'LAKSHADWEEP'
 'APO' 'PONDICHERRY' 'RAJSTHAN']


In [ ]:
ds = ds.drop(['Fulfilled_By', 'Promotion_IDs'], axis=1)
ds.loc[(ds['Courier_Status'].isna()) & (ds['Quantity'] == 0), 'Courier_Status'] = 'Cancelled'
ds['Currency'] = ds['Currency'].fillna('INR')

In [ ]:
ds['Sale_Amount'] = ds.groupby('Product_SKU')['Sale_Amount'].transform(
    lambda x: x.fillna(method='bfill').fillna(method='ffill')
)

imputer = SimpleImputer(strategy='median')
ds['Sale_Amount'] = imputer.fit_transform(ds[['Sale_Amount']])
ds.info()

<ipython-input-28-d48ff32f1275>:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lambda x: x.fillna(method='bfill').fillna(method='ffill')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   index                   128975 non-null  int64  
 1   Order_ID                128975 non-null  object 
 2   Order_Date              128975 non-null  object 
 3   Order_Status            128975 non-null  object 
 4   Fulfillment_Type        128975 non-null  object 
 5   Sales_Channel           128975 non-null  object 
 6   Shipping_Service_Level  128975 non-null  object 
 7   Product_Style           128975 non-null  object 
 8   Product_SKU             128975 non-null  object 
 9   Product_Category        128975 non-null  object 
 10  Product_Size            128975 non-null  object 
 11  Amazon_Standard_ID      128975 non-null  object 
 12  Courier_Status          128975 non-null  object 
 13  Quantity                128975 non-null  int64  
 14  Currency            

International Sale Report cleaning

In [ ]:
ds2 = pd.read_csv("/content/International sale Report.csv")
print(ds2.shape)
ds2.info()

(37432, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37432 entries, 0 to 37431
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   index               37432 non-null  int64 
 1   Sale_Date           37431 non-null  object
 2   Sale_Month          37407 non-null  object
 3   Customer_Name       36392 non-null  object
 4   Product_Style       36392 non-null  object
 5   Product_SKU         34958 non-null  object
 6   Product_Size        36392 non-null  object
 7   Quantity_Purchased  36392 non-null  object
 8   Price_per_Unit      36392 non-null  object
 9   Gross_Amount        36392 non-null  object
dtypes: int64(1), object(9)
memory usage: 2.9+ MB


In [ ]:
most_frequent_sku = ds2.groupby('Product_Style')['Product_SKU'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
ds2.loc[ds2['Product_SKU'].isna(), 'Product_SKU'] = ds2.loc[ds2['Product_SKU'].isna(), 'Product_Style'].map(most_frequent_sku)
ds2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37432 entries, 0 to 37431
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   index               37432 non-null  int64 
 1   Sale_Date           37431 non-null  object
 2   Sale_Month          37407 non-null  object
 3   Customer_Name       36392 non-null  object
 4   Product_Style       36392 non-null  object
 5   Product_SKU         36189 non-null  object
 6   Product_Size        36392 non-null  object
 7   Quantity_Purchased  36392 non-null  object
 8   Price_per_Unit      36392 non-null  object
 9   Gross_Amount        36392 non-null  object
dtypes: int64(1), object(9)
memory usage: 2.9+ MB
